In [1]:
import numpy as np
from pynq import Overlay
from pynq import allocate
import cv2

In [2]:
overlay = Overlay("/home/root/jupyter_notebooks/traffic_signal_recognition/design_1.bit")
dma = overlay.axi_dma_0
traffic_light_recognition = overlay.traffic_light_recogn_0

In [3]:
def recognition(input_data, in_buffer):
    in_buffer[:,:,0:3] = input_data[:]
    dma.sendchannel.transfer(in_buffer)
    traffic_light_recognition.write(0x00, 0x01)
    dma.sendchannel.wait()
    while not (traffic_light_recognition.read(0x00) >> 1) & 0x01:
        continue
    #return traffic_light_recognition.read(0x28)

In [4]:
traffic_light_recognition.register_map

RegisterMap {
  CTRL = Register(AP_START=0, AP_DONE=0, AP_IDLE=1, AP_READY=0, RESERVED_1=0, AUTO_RESTART=0, RESERVED_2=0, INTERRUPT=0, RESERVED_3=0),
  GIER = Register(Enable=0, RESERVED=0),
  IP_IER = Register(CHAN0_INT_EN=0, CHAN1_INT_EN=0, RESERVED_0=0),
  IP_ISR = Register(CHAN0_INT_ST=0, CHAN1_INT_ST=0, RESERVED_0=0),
  rows = Register(rows=write-only),
  cols = Register(cols=write-only),
  radius = Register(radius=write-only),
  Memory_cond_green = Register(value=0),
  Memory_cond_yellow = Register(value=0),
  Memory_cond_red = Register(value=0),
  circle_center_x = Register(circle_center_x=0),
  circle_center_x_ctrl = Register(circle_center_x_ap_vld=0, RESERVED=0),
  circle_center_y = Register(circle_center_y=0),
  circle_center_y_ctrl = Register(circle_center_y_ap_vld=0, RESERVED=0),
  circle_radius = Register(circle_radius=0),
  circle_radius_ctrl = Register(circle_radius_ap_vld=0, RESERVED=0),
  color = Register(color=0),
  color_ctrl = Register(color_ap_vld=0, RESERVED=0)
}

In [5]:
traffic_green = "../image/green_256_128.jpg"
traffic_yellow = "../image/yellow_256_128.jpg"
traffic_red = "../image/red_256_128.jpg"

images = [traffic_green, traffic_yellow, traffic_red]

for i, name in enumerate(images):
    print(name)
    input_img = cv2.imread(name)
    height = input_img.shape[0]
    width = input_img.shape[1]
    channel = input_img.shape[2] + 1

    in_buffer = allocate(shape=(height, width, channel), dtype=np.uint8)
    
    traffic_light_recognition.write(0x10, input_img.shape[0]) # rows
    traffic_light_recognition.write(0x18, input_img.shape[1]) # cols
    traffic_light_recognition.write(0x20, 20) # radius
    
    traffic_light_recognition.write(48, 150) # cond_green
    traffic_light_recognition.write(52, 200) # 
    traffic_light_recognition.write(56, 150) # 
    
    traffic_light_recognition.write(64, 200) # cond_yellow
    traffic_light_recognition.write(68, 150) # 
    traffic_light_recognition.write(72, 150) # 
    
    traffic_light_recognition.write(80, 200) # cond_red
    traffic_light_recognition.write(84, 150) # 
    traffic_light_recognition.write(88, 150) # 
    recognition(input_img, in_buffer)
    
    x = traffic_light_recognition.read(0x60) # circle_center_x
    y = traffic_light_recognition.read(0x70) # circle_center_y
    r = traffic_light_recognition.read(0x80) # circle_radius
    color = traffic_light_recognition.read(0x90) # color
    
    print(f"x={x}, y={y}, r={r}")
    print(color)

./image/green_256_128.jpg
x=52, y=43, r=20
3
./image/yellow_256_128.jpg
x=124, y=41, r=20
2
./image/red_256_128.jpg
x=181, y=43,r=20
1
